In [1]:
from pylsl import StreamInlet, resolve_stream
import matplotlib.pyplot as plt
import numpy as np
from joblib import dump, load

from ahk import AHK
import keyboard
import threading
import time
import sys

ahk = AHK()
buffer = np.full((750, 4), sys.maxsize)
lock = threading.Lock()
clf = load('./models/mlp2.joblib')

def start_stream():
    print("looking for an EEG stream...")
    streams = resolve_stream('type', 'EEG')
    inlet = StreamInlet(streams[0])
    while True:
        chunk,ts = inlet.pull_sample()
        chunk = np.array(chunk)[0: 4]
        with lock:
            buffer[:-1] = buffer[1:]
            buffer[-1:] = [chunk]
    
def get_data():
    with lock:
        temp = buffer
        return temp

def keyboard_control(y_pred, toggle):
    def release_all():
        ahk.key_up("a")
        ahk.key_up("z")
        ahk.key_up(",")
        ahk.key_up(".")

    if keyboard.is_pressed("esc"):
        release_all()
        print("quit")
        sys.exit()
        return
    # jaw clenched controls accelerate/decelerate
    if int(y_pred[0]) == 3:
        ahk.key_down("a")
    # left blink controls left turn
    elif int(y_pred[0]) == 1:
        ahk.key_down(",")
    # right blink controls right turn
    elif int(y_pred[0]) == 2:
        ahk.key_down(".")
    elif int(y_pred[0]) == 0:
        ahk.key_down("z")
    time.sleep(0.5)
    release_all()
    return toggle
    
# Start steam
th = threading.Thread(target = start_stream)
th.start()
toggle = True

while True:
    if keyboard.is_pressed("esc"):
        print("exiting program")
        break

    # gets data from sliding stream thread
    buffer = get_data()
    model_feed_data = np.array([buffer.T])
    y_pred = clf.predict(model_feed_data)
    print(y_pred)
    time.sleep(0.1)
    # steers car using model's prediction
    #toggle = keyboard_control(y_pred, toggle)
    

print('Streams closed')
            

looking for an EEG stream...
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[1.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[3.]
[3.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[1.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[1.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[3.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[3.]
[1.]
[3.]
[3.]
[3.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[

KeyboardInterrupt: 